# Welcome to the September 2021 Tabular Playground Competition! #

In this competition, we predict whether a customer will make an insurance claim.

# Data #

The full dataset has almost one million rows. We'll use just a sample so we can explore the data more quickly.

In [1]:
import pandas as pd
from pathlib import Path

data_dir = Path('../input/tabular-playground-series-sep-2021/')

df_train = pd.read_csv(
    data_dir / "train.csv",
    index_col='id',
    nrows=25000,  # comment this row to use the full dataset
)

FEATURES = df_train.columns[:-1]
TARGET = df_train.columns[-1]

df_train.head()

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,f110,f111,f112,f113,f114,f115,f116,f117,f118,claim
id,,,,,,,,,,,,,,,,,,,,,
0,0.10859,0.004314,-37.566,0.017364,0.28915,-10.25100,135.12,168900.0,3.992400e+14,86.489,...,-12.2280,1.7482,1.90960,-7.11570,4378.80,1.2096,8.613400e+14,140.1,1.01770,1
1,0.10090,0.299610,11822.000,0.276500,0.45970,-0.83733,1721.90,119810.0,3.874100e+15,9953.600,...,-56.7580,4.1684,0.34808,4.14200,913.23,1.2464,7.575100e+15,1861.0,0.28359,0
2,0.17803,-0.006980,907.270,0.272140,0.45948,0.17327,2298.00,360650.0,1.224500e+13,15827.000,...,-5.7688,1.2042,0.26290,8.13120,45119.00,1.1764,3.218100e+14,3838.2,0.40690,1
3,0.15236,0.007259,780.100,0.025179,0.51947,7.49140,112.51,259490.0,7.781400e+13,-36.837,...,-34.8580,2.0694,0.79631,-16.33600,4952.40,1.1784,4.533000e+12,4889.1,0.51486,1
4,0.11623,0.502900,-109.150,0.297910,0.34490,-0.40932,2538.90,65332.0,1.907200e+15,144.120,...,-13.6410,1.5298,1.14640,-0.43124,3856.50,1.4830,-8.991300e+12,NaN,0.23049,1


The target `'claim'` has binary outcomes: `0` for no claim and `1` for claim.

# Model #

Let's try out a simple XGBoost model. This algorithm can handle missing values, but you could try imputing them instead.  We use `XGBClassifier` (instead of `XGBRegressor`, for instance), since this is a classification problem.

In [2]:
from xgboost import XGBClassifier

X = df_train.loc[:, FEATURES]
y = df_train.loc[:, TARGET]

model = XGBClassifier(
    max_depth=3,
    subsample=0.5,
    colsample_bytree=0.5,
    n_jobs=-1,
    # Uncomment if you want to use GPU. Recommended for whole training set.
    #tree_method='gpu_hist',
    random_state=0,
)


# Evaluation #

The evaluation metric is AUC, which stands for "area under curve".  Run the next code cell to evaluate the model.

In [3]:
from sklearn.model_selection import cross_validate
import warnings 
warnings.filterwarnings('ignore')

def score(X, y, model, cv):
    scoring = ["roc_auc"]
    scores = cross_validate(
        model, X, y, scoring=scoring, cv=cv, return_train_score=True
    )
    scores = pd.DataFrame(scores).T
    return scores.assign(
        mean = lambda x: x.mean(axis=1),
        std = lambda x: x.std(axis=1),
    )

scores = score(X, y, model, cv=2)

display(scores)

[15:22:27] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:22:32] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


,0,1,mean,std
fit_time,5.163640,5.763061,5.463351,0.299710
score_time,0.027473,0.027307,0.027390,0.000083
test_roc_auc,0.758076,0.751630,0.754853,0.003223
train_roc_auc,0.867126,0.868199,0.867662,0.000536


A "neutral" AUC is 0.5, so anything better than that means our model learned something useful.

# Make Submission #

Our predictions are binary 0 and 1, but you're allowed to submit probabilities instead. In scikit-learn, you would use the `predict_proba` method instead of `predict`.

In [4]:
# Fit on full training set
model.fit(X, y)

X_test = pd.read_csv(data_dir / "test.csv", index_col='id')

# Make predictions
y_pred = pd.Series(
    model.predict(X_test),
    index=X_test.index,
    name=TARGET,
)

# Create submission file
y_pred.to_csv("submission.csv")

[15:22:38] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
